In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from joblib import Parallel, delayed
from pathlib import Path
from scipy.signal import savgol_filter
from tqdm import tqdm
from numba import jit

from utils.data import PRED_SINK, ALL_B_COLS, ALL_H_COLS
from utils.experiments import get_bh_integral_from_two_mats
from run_cnn_inference import run_inference
from utils.data import load_material_csv_files_and_generate_pandas_df

plt.style.use("default")
plt.rcParams.update({
    #"text.usetex": False,
    "font.family": "serif",
    #"font.serif": ["Helvetica"],
    'figure.dpi': 200,  # renders images larger for notebook
    'axes.grid': True,
    'grid.alpha': 0.3,

})
IMG_PATH = Path.cwd().parent.parent / 'notebooks' / 'img'
mat2mat_d = {'A': "3C92", "B": "T37", "C": "3C95", "D": "79", "E": "ML95S"}
mat_lbl_l = list(mat2mat_d.values())

final_train_raw = Path.cwd().parent.parent / 'Model' /'data' / 'input' / 'raw' / 'final-training'
final_test_raw = Path.cwd().parent.parent / 'Model' / 'data' / 'input'/ 'raw' / 'final-testing' 


In [2]:
@jit(nopython=True)
def stenglein_mdl(theta, observations):
    """theta = a1, a2, a3, a4, k, gamma"""
    frequency = observations[:, 0]
    b_deltas = observations[:, 1:5]
    integral_part = observations[:, 5]
    w_hyst = b_deltas @ theta[:4].reshape(-1, 1)
    f_eff = frequency * (1 + theta[4] * integral_part ** theta[5])
    p_loss = w_hyst * f_eff
    return p_loss

@jit(nopython=True)
def stenglein_loss(theta, observations):
    ploss_hat = stenglein_mdl(theta, observations[:, :-1])
    msle = np.mean((np.log(np.clip(ploss_hat, 1, None)) - np.log(observations[:, -1]))**2)
    return msle

In [7]:
mat_sat_b = {}

def construct_features(path, mat_lbl, temperature=25):
    df = load_material_csv_files_and_generate_pandas_df(path, training=True)
    if temperature is not None:
        df = df.query(f"temp == {temperature}")
    b_field = df.loc[:, ALL_B_COLS].to_numpy()

    

    delta_b_raw = (b_field.max(axis=1) - b_field.min(axis=1))
    # cache saturation constant
    if mat_lbl not in mat_sat_b:
        b_delta_max = 0.9*delta_b_raw.max()  # maximum b across mat data
        mat_sat_b[mat_lbl] = b_delta_max
    else:
        b_delta_max = mat_sat_b[mat_lbl]
    delta_b = delta_b_raw / b_delta_max
    delta_b2 = delta_b**2
    delta_b3 = delta_b**3
    delta_b4 = delta_b**4

    # derivatives
    b_deriv = np.empty((b_field.shape[0], b_field.shape[1] + 2))
    b_field_smooth = savgol_filter(b_field, 50, 3, axis=-1, mode='wrap')
    b_deriv[:, 1:-1] = b_field_smooth
    b_deriv[:, 0] = b_field_smooth[:, -1]
    b_deriv[:, -1] = b_field_smooth[:, 0]
    b_deriv = np.gradient(b_deriv, axis=1) * df.loc[:, ['freq']].to_numpy()
    b_deriv_sq = np.gradient(b_deriv, axis=1) * df.loc[:, ['freq']].to_numpy()
    #b_deriv = b_deriv[:, 1:-1]
    b_deriv_sq = b_deriv_sq[:, 1:-1]
    
    b_deriv_second_l1_loss = np.abs(b_deriv_sq).sum(axis=1)
    integral_part = b_deriv_second_l1_loss / delta_b_raw
    relevant_df = pd.DataFrame({'freq': df.freq, 'delta_b': delta_b, 'delta_b2': delta_b2, 'delta_b3': delta_b3, 'delta_b4': delta_b4, 'integral_part': integral_part, 'ploss': df.ploss})
    return relevant_df.to_numpy()

## Training

In [8]:
gt_train_d = {}
gt_test_d = {}
for mat_lbl in mat_lbl_l:
    gt_train_d[mat_lbl] = construct_features(final_train_raw / mat_lbl, mat_lbl)
    gt_test_d[mat_lbl] = construct_features(final_test_raw / mat_lbl, mat_lbl)

In [9]:
x0 = np.random.randn(6)
x0[-1] = np.abs(x0[-1] + 1)*0.1
with Parallel(n_jobs=5) as prll:
    opt_results = prll(delayed(minimize)(stenglein_loss, x0, gt_train_d[mat_lbl], method='BFGS') for mat_lbl in mat_lbl_l)


/tmp/ipykernel_17272/3767990119.py:14: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
/tmp/ipykernel_17272/3767990119.py:14: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
/tmp/ipykernel_17272/3767990119.py:14: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
/tmp/ipykernel_17272/3767990119.py:14: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
/tmp/ipykernel_17272/3767990119.py:14: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=Tr

In [10]:
rel_errs_l = []
for mat_lbl, opt_res in zip(mat_lbl_l, opt_results):
    print(mat_lbl)
    print(opt_res)
    # train
    mat_train_gt = gt_train_d[mat_lbl]
    rel_errs_train = (stenglein_mdl(opt_res.x, mat_train_gt[:, :-1]) - mat_train_gt[:, -1]) / mat_train_gt[:, -1]
    avg_rel_err_train = np.mean(rel_errs_train)
    rel_err_95th_train = np.quantile(rel_errs_train, 0.95)
    rel_err_99th_train = np.quantile(rel_errs_train, 0.99)
    
    # test
    mat_test_gt = gt_test_d[mat_lbl]
    rel_errs_test = (stenglein_mdl(opt_res.x, mat_test_gt[:, :-1]) - mat_test_gt[:, -1]) / mat_test_gt[:, -1]
    avg_rel_err_test = np.mean(rel_errs_test)
    rel_err_95th_test = np.quantile(rel_errs_test, 0.95)
    rel_err_99th_test = np.quantile(rel_errs_test, 0.99)

    rel_errs_l.append(pd.Series({'material': mat_lbl,
                                 'avg_rel_err_train': avg_rel_err_train, 'rel_err_95th_train': rel_err_95th_train, 'rel_err_99th_train': rel_err_99th_train,
                                 'avg_rel_err_test': avg_rel_err_test, 'rel_err_95th_test': rel_err_95th_test, 'rel_err_99th_test': rel_err_99th_test, }))
pd.DataFrame(rel_errs_l)

3C92
  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 3.4743681768490817
        x: [-1.093e+01  4.431e+01 -6.325e+01  2.869e+01 -2.032e+01
            -1.169e-01]
      nit: 70
      jac: [-3.096e-05  1.624e-04  2.957e-04  4.151e-04  7.012e-05
             1.023e-02]
 hess_inv: [[ 8.606e+02 -3.435e+03 ...  1.250e+03  5.068e+00]
            [-3.435e+03  1.440e+04 ... -5.670e+03 -2.160e+01]
            ...
            [ 1.250e+03 -5.670e+03 ...  2.486e+03  8.745e+00]
            [ 5.068e+00 -2.160e+01 ...  8.745e+00  3.311e-02]]
     nfev: 985
     njev: 139
T37
  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 2.6471126284962825
        x: [-6.851e+00  2.498e+01 -3.320e+01  1.441e+01 -1.676e+01
            -1.059e-01]
      nit: 85
      jac: [ 2.584e-05  4.280e-05  1.380e-05  2.909e-05  2.304e-05
             2.947e-04]
 hess_inv: [[ 1.186e+00  4.147e-02 ...

,material,avg_rel_err_train,rel_err_95th_train,rel_err_99th_train,avg_rel_err_test,rel_err_95th_test,rel_err_99th_test
0,3C92,3.261780,18.663940,35.048256,3.226474,16.780819,38.315445
1,T37,2.257100,11.349432,22.580281,2.330231,12.081121,25.777880
2,3C95,3.212853,18.057030,34.878556,3.840443,18.607604,43.694991
3,79,2.641042,15.406743,31.398647,2.249629,15.674245,42.371725
4,ML95S,3.488473,21.165083,45.205670,2.844742,14.336703,36.396225
